In [13]:
import pandas as pd
import os
# import sys to get more detailed Python exception info
import sys
# import the connect library for psycopg2
import psycopg2
# import the error handling libraries for psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras
import pandas as pd
from io import StringIO
import numpy as np
from timeit import default_timer as timer
import postgressql
import mlflow
import tempfile

In [7]:
conn_params_dic = {
    "host"      : "localhost",
    "database"  : "bank_raw_data",
    "user"      : "mlflow_user"
}

os.environ['MLFLOW_TRACKING_URI'] = "http://0.0.0.0:8000"
os.environ['MLFLOW_TRACKING_USERNAME'] = "Pablo"

In [8]:
client = mlflow.tracking.MlflowClient()

In [11]:
mlflow.set_experiment("bankchurners_etl")

2022/05/30 23:53:29 INFO mlflow.tracking.fluent: Experiment with name 'bankchurners_etl' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/salv91/Desktop/mlflow_artifactRoot/mlruns/1', experiment_id='1', lifecycle_stage='active', name='bankchurners_etl', tags={}>

In [19]:
with mlflow.start_run() as run:
    
    connection=postgressql.connect(conn_params_dic)
    column_names=['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']

    table_name = "users"

    df = postgressql.query_read(connection,column_names=column_names,table_name = "users")
    
    temp = tempfile.NamedTemporaryFile(prefix="Raw_Data_", suffix=".csv")
    temp_name = temp.name
    
    try:
        df.to_csv(temp_name, index=False)
        mlflow.log_artifact(temp_name, "Raw_Data")
    finally:
        temp.close() # Delete the temp file

    #print info run
    run_id = run.info.run_id
    mlflow.set_tag("run_id", run_id)
    print(run.info)
    mlflow.end_run()

Connecting to the PostgreSQL...........
Connection successfully..................
<RunInfo: artifact_uri='/Users/salv91/Desktop/mlflow_artifactRoot/mlruns/1/dd03e24808ee404296c929ee7c13a63b/artifacts', end_time=None, experiment_id='1', lifecycle_stage='active', run_id='dd03e24808ee404296c929ee7c13a63b', run_uuid='dd03e24808ee404296c929ee7c13a63b', start_time=1653973322616, status='RUNNING', user_id='salv91'>
